In [ ]:
import re, nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('omw-1.4')
nltk.download('wordnet')
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
import joblib

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
import re, nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('omw-1.4')
nltk.download('wordnet')
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
import joblib
import pandas as pd
import json


"""
The dataset had to be downloaded in two different parts because parsehub only lets you download 10,000 pages each time you use it
so df2 and df3 have to be combined into one dataset here in colab
"""

df2 = pd.read_csv("/content/drive/MyDrive/Donedeal_Data_1.csv")
df3 = pd.read_csv("/content/drive/MyDrive/Donedeal_Data_2.csv")


display(df2.head())
display(df3.head())



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,name,url,Trim,Mileage,Engine,NCT_Expiry,Year,Transmission,Total_Owners,Make,Fuel_Type,Body_Type,Doors,Trim_Level,Seats,Colour,Road_Tax,Rural_Fuel_Consumption,Comb_Fuel_Consumption,Urban_Fuel_Consumption,Engine_Size,External_Length,External_Height,External_Width,Euro_NCAP_Overall_Safety_Rating,Child_Protection_Rating,Safety_Assistance_Rating,Adult_Protection_Rating,Pedestrian_Protection_Rating,Current_Country_Of_Reg,Imported,Independant_Dealer,Dublin2,Model,Price,Boot_Volume,Ground_Clearance,County,Fuel_Tank_Capacity,Engine_Power,Top_Speed,Battery_Capacity,Acceleration,section
0,Hyundai Tucson 2016 Executive,https://www.donedeal.ie/cars-for-sale/hyundai-...,Executive,"128,000 mi",1.7 L Diesel,Jun 2026,2016,Manual,2,Hyundai,Diesel,SUV,5,Medium Trim,5,Black,200.0,4.0 L/100 km,4.4 L/100 km,5.1 L/100 km,1.7 L,4.48 m,1.65 m,1.85 m,5/5,85%,71%,86%,71%,Ireland,No,Private Seller\nNo rating yet,"Ardara, Co. Donegal",Tucson,"€10,500",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,selection1
1,Audi Q5 40 TDI Quattro S Line 5dr S Tronic,https://www.donedeal.ie/cars-for-sale/audi-q5-...,---,"45,139 mi",2.0 L Diesel,---,2021,Automatic,---,Audi,Diesel,---,---,---,5,White,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UK,NaN,Franchise Dealer,"Dungannon, Co. Tyrone",Q5,"£27,740",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,selection1
2,Volkswagen Passat 2013 - New NCT,https://www.donedeal.ie/cars-for-sale/volkswag...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,selection1
3,2018 AUDI A6 DIESEL SALOON 2.0 TDI ULTRA S- LINE,https://www.donedeal.ie/cars-for-sale/2018-aud...,S Line,"81,300 mi",2.0 L Diesel,Jun 2026,2018,Manual,1,Audi,Diesel,Saloon,4,Sports Trim,5,Grey,200.0,3.9 L/100 km,4.4 L/100 km,5.2 L/100 km,2.0 L,4.93 m,1.46 m,1.87 m,5/5,83%,86%,91%,41%,Ireland,UK,Verified Trader\nNo rating yet,"Derry City, Co. Derry",A6,"€22,500",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,selection1
4,Mercedes-Benz E-Class E220 D Coupe AMG Sport,https://www.donedeal.ie/cars-for-sale/mercedes...,AMG SPORT,"137,000 km",2.0 L Diesel,Apr 2026,2018,Automatic,4,Mercedes-Benz,Diesel,Coupe,2,Sports Trim,4,Blue,200.0,3.7 L/100 km,4.0 L/100 km,4.5 L/100 km,2.0 L,4.83 m,1.43 m,1.86 m,5/5,90%,62%,95%,77%,Ireland,No,Independent Dealer,"Nenagh, Co. Tipperary",E-Class,"€28,950",425 L,10 cm,Co. Tipperary,NaN,NaN,NaN,NaN,NaN,selection1


,name,url,Trim,Mileage,Engine,NCT_Expiry,Year,Transmission,Total_Owners,Make,Fuel_Type,Body_Type,Doors,Trim_Level,Seats,Colour,Road_Tax,Current_Country_Of_Reg,Imported,County,Independant_Dealer,Dublin2,Model,Price,Engine_Size,External_Length,Fuel_Tank_Capacity,External_Width,Rural_Fuel_Consumption,Comb_Fuel_Consumption,Urban_Fuel_Consumption,External_Height,Boot_Volume,Ground_Clearance,Euro_NCAP_Overall_Safety_Rating,Engine_Power,Top_Speed,Acceleration,Child_Protection_Rating,Safety_Assistance_Rating,Adult_Protection_Rating,Pedestrian_Protection_Rating,Battery_Capacity
0,2016 Audi A3 1.4TFSI AUTO SLINE SPEC 1YR WARRANTY,https://www.donedeal.ie/cars-for-sale/2016-aud...,S Line,"128,129 km",1.4 L Petrol,May 2027,2016,Automatic,1,Audi,Petrol,Saloon,4,---,5,White,0.0,Ireland,Unknown,Co. Dublin,Independent Dealer,Co. Dublin,A3,"€16,890",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BMW 2 Series 218D Auto Sport Gran Coupe,https://www.donedeal.ie/cars-for-sale/bmw-2-se...,Sport,"140,000 km",2.0 L Diesel,Feb 2027,2021,Automatic,1,BMW,Diesel,Saloon,4,Base Trim,5,Grey,NaN,Ireland,No,Co. Dublin,Independent Dealer,Co. Dublin,2-Series,"€25,750",2.0 L,4.53 m,42 L,1.80 m,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"BMW 118i SPORT AUTO, 2017",https://www.donedeal.ie/cars-for-sale/bmw-118i...,Sport,"32,000 km",1.6 L Petrol,Sep 2026,2017,Automatic,1,BMW,Petrol,Hatchback,5,Sports Trim,5,White,270.0,Ireland,Unknown,Co. Monaghan,Independent Dealer,"Castleblaney, Co. Monaghan",1-Series,"€18,250",1.6 L,4.33 m,52 L,1.77 m,4.5 L/100 km,5.3 L/100 km,6.5 L/100 km,1.44 m,"1,200 L",14 cm,5/5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Hyundai Ioniq 5 AWD 73kw Premium Plus awd call Jo,https://www.donedeal.ie/cars-for-sale/hyundai-...,Premium Plus,"44,410 km",430 km (WLTP) Electric,Feb 2026,2022,Automatic,1,Hyundai,Electric,MPV,5,Luxury Trim,5,Black,120.0,Ireland,No,NaN,Franchise Dealer,"Sandyford, Co. Dublin",IONIQ 5,"€34,950",NaN,4.64 m,NaN,1.89 m,NaN,NaN,NaN,NaN,NaN,NaN,NaN,306 hp,185 km/h,5.2 sec,NaN,NaN,NaN,NaN,NaN
4,Hyundai INSTER Elegance 49kw,https://www.donedeal.ie/cars-for-sale/hyundai-...,---,0 km,Electric,---,2025,Automatic,---,NaN,Electric,Hatchback,5,---,---,---,NaN,NaN,NaN,Co. Dublin,Franchise Dealer,"Swords, Co. Dublin",INSTER,"€23,550",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
pd.set_option('display.max_columns', None)

df2.head()
df3.head()

,name,url,Trim,Mileage,Engine,NCT_Expiry,Year,Transmission,Total_Owners,Make,Fuel_Type,Body_Type,Doors,Trim_Level,Seats,Colour,Road_Tax,Current_Country_Of_Reg,Imported,County,Independant_Dealer,Dublin2,Model,Price,Engine_Size,External_Length,Fuel_Tank_Capacity,External_Width,Rural_Fuel_Consumption,Comb_Fuel_Consumption,Urban_Fuel_Consumption,External_Height,Boot_Volume,Ground_Clearance,Euro_NCAP_Overall_Safety_Rating,Engine_Power,Top_Speed,Acceleration,Child_Protection_Rating,Safety_Assistance_Rating,Adult_Protection_Rating,Pedestrian_Protection_Rating,Battery_Capacity
0,2016 Audi A3 1.4TFSI AUTO SLINE SPEC 1YR WARRANTY,https://www.donedeal.ie/cars-for-sale/2016-aud...,S Line,"128,129 km",1.4 L Petrol,May 2027,2016,Automatic,1,Audi,Petrol,Saloon,4,---,5,White,0.0,Ireland,Unknown,Co. Dublin,Independent Dealer,Co. Dublin,A3,"€16,890",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BMW 2 Series 218D Auto Sport Gran Coupe,https://www.donedeal.ie/cars-for-sale/bmw-2-se...,Sport,"140,000 km",2.0 L Diesel,Feb 2027,2021,Automatic,1,BMW,Diesel,Saloon,4,Base Trim,5,Grey,NaN,Ireland,No,Co. Dublin,Independent Dealer,Co. Dublin,2-Series,"€25,750",2.0 L,4.53 m,42 L,1.80 m,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"BMW 118i SPORT AUTO, 2017",https://www.donedeal.ie/cars-for-sale/bmw-118i...,Sport,"32,000 km",1.6 L Petrol,Sep 2026,2017,Automatic,1,BMW,Petrol,Hatchback,5,Sports Trim,5,White,270.0,Ireland,Unknown,Co. Monaghan,Independent Dealer,"Castleblaney, Co. Monaghan",1-Series,"€18,250",1.6 L,4.33 m,52 L,1.77 m,4.5 L/100 km,5.3 L/100 km,6.5 L/100 km,1.44 m,"1,200 L",14 cm,5/5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Hyundai Ioniq 5 AWD 73kw Premium Plus awd call Jo,https://www.donedeal.ie/cars-for-sale/hyundai-...,Premium Plus,"44,410 km",430 km (WLTP) Electric,Feb 2026,2022,Automatic,1,Hyundai,Electric,MPV,5,Luxury Trim,5,Black,120.0,Ireland,No,NaN,Franchise Dealer,"Sandyford, Co. Dublin",IONIQ 5,"€34,950",NaN,4.64 m,NaN,1.89 m,NaN,NaN,NaN,NaN,NaN,NaN,NaN,306 hp,185 km/h,5.2 sec,NaN,NaN,NaN,NaN,NaN
4,Hyundai INSTER Elegance 49kw,https://www.donedeal.ie/cars-for-sale/hyundai-...,---,0 km,Electric,---,2025,Automatic,---,NaN,Electric,Hatchback,5,---,---,---,NaN,NaN,NaN,Co. Dublin,Franchise Dealer,"Swords, Co. Dublin",INSTER,"€23,550",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:

#Parsehub automatically added selection1_ to the beginning of each column name so I am removing it here
df2.columns = df2.columns.str.replace('^selection1_', '', regex=True)
df3.columns = df3.columns.str.replace('^selection1_', '', regex=True)
display(df2.head())
display(df3.head())







,name,url,Trim,Mileage,Engine,NCT_Expiry,Year,Transmission,Total_Owners,Make,Fuel_Type,Body_Type,Doors,Trim_Level,Seats,Colour,Road_Tax,Rural_Fuel_Consumption,Comb_Fuel_Consumption,Urban_Fuel_Consumption,Engine_Size,External_Length,External_Height,External_Width,Euro_NCAP_Overall_Safety_Rating,Child_Protection_Rating,Safety_Assistance_Rating,Adult_Protection_Rating,Pedestrian_Protection_Rating,Current_Country_Of_Reg,Imported,Independant_Dealer,Dublin2,Model,Price,Boot_Volume,Ground_Clearance,County,Fuel_Tank_Capacity,Engine_Power,Top_Speed,Battery_Capacity,Acceleration,section
0,Hyundai Tucson 2016 Executive,https://www.donedeal.ie/cars-for-sale/hyundai-...,Executive,"128,000 mi",1.7 L Diesel,Jun 2026,2016,Manual,2,Hyundai,Diesel,SUV,5,Medium Trim,5,Black,200.0,4.0 L/100 km,4.4 L/100 km,5.1 L/100 km,1.7 L,4.48 m,1.65 m,1.85 m,5/5,85%,71%,86%,71%,Ireland,No,Private Seller\nNo rating yet,"Ardara, Co. Donegal",Tucson,"€10,500",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,selection1
1,Audi Q5 40 TDI Quattro S Line 5dr S Tronic,https://www.donedeal.ie/cars-for-sale/audi-q5-...,---,"45,139 mi",2.0 L Diesel,---,2021,Automatic,---,Audi,Diesel,---,---,---,5,White,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UK,NaN,Franchise Dealer,"Dungannon, Co. Tyrone",Q5,"£27,740",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,selection1
2,Volkswagen Passat 2013 - New NCT,https://www.donedeal.ie/cars-for-sale/volkswag...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,selection1
3,2018 AUDI A6 DIESEL SALOON 2.0 TDI ULTRA S- LINE,https://www.donedeal.ie/cars-for-sale/2018-aud...,S Line,"81,300 mi",2.0 L Diesel,Jun 2026,2018,Manual,1,Audi,Diesel,Saloon,4,Sports Trim,5,Grey,200.0,3.9 L/100 km,4.4 L/100 km,5.2 L/100 km,2.0 L,4.93 m,1.46 m,1.87 m,5/5,83%,86%,91%,41%,Ireland,UK,Verified Trader\nNo rating yet,"Derry City, Co. Derry",A6,"€22,500",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,selection1
4,Mercedes-Benz E-Class E220 D Coupe AMG Sport,https://www.donedeal.ie/cars-for-sale/mercedes...,AMG SPORT,"137,000 km",2.0 L Diesel,Apr 2026,2018,Automatic,4,Mercedes-Benz,Diesel,Coupe,2,Sports Trim,4,Blue,200.0,3.7 L/100 km,4.0 L/100 km,4.5 L/100 km,2.0 L,4.83 m,1.43 m,1.86 m,5/5,90%,62%,95%,77%,Ireland,No,Independent Dealer,"Nenagh, Co. Tipperary",E-Class,"€28,950",425 L,10 cm,Co. Tipperary,NaN,NaN,NaN,NaN,NaN,selection1


,name,url,Trim,Mileage,Engine,NCT_Expiry,Year,Transmission,Total_Owners,Make,Fuel_Type,Body_Type,Doors,Trim_Level,Seats,Colour,Road_Tax,Current_Country_Of_Reg,Imported,County,Independant_Dealer,Dublin2,Model,Price,Engine_Size,External_Length,Fuel_Tank_Capacity,External_Width,Rural_Fuel_Consumption,Comb_Fuel_Consumption,Urban_Fuel_Consumption,External_Height,Boot_Volume,Ground_Clearance,Euro_NCAP_Overall_Safety_Rating,Engine_Power,Top_Speed,Acceleration,Child_Protection_Rating,Safety_Assistance_Rating,Adult_Protection_Rating,Pedestrian_Protection_Rating,Battery_Capacity
0,2016 Audi A3 1.4TFSI AUTO SLINE SPEC 1YR WARRANTY,https://www.donedeal.ie/cars-for-sale/2016-aud...,S Line,"128,129 km",1.4 L Petrol,May 2027,2016,Automatic,1,Audi,Petrol,Saloon,4,---,5,White,0.0,Ireland,Unknown,Co. Dublin,Independent Dealer,Co. Dublin,A3,"€16,890",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BMW 2 Series 218D Auto Sport Gran Coupe,https://www.donedeal.ie/cars-for-sale/bmw-2-se...,Sport,"140,000 km",2.0 L Diesel,Feb 2027,2021,Automatic,1,BMW,Diesel,Saloon,4,Base Trim,5,Grey,NaN,Ireland,No,Co. Dublin,Independent Dealer,Co. Dublin,2-Series,"€25,750",2.0 L,4.53 m,42 L,1.80 m,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"BMW 118i SPORT AUTO, 2017",https://www.donedeal.ie/cars-for-sale/bmw-118i...,Sport,"32,000 km",1.6 L Petrol,Sep 2026,2017,Automatic,1,BMW,Petrol,Hatchback,5,Sports Trim,5,White,270.0,Ireland,Unknown,Co. Monaghan,Independent Dealer,"Castleblaney, Co. Monaghan",1-Series,"€18,250",1.6 L,4.33 m,52 L,1.77 m,4.5 L/100 km,5.3 L/100 km,6.5 L/100 km,1.44 m,"1,200 L",14 cm,5/5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Hyundai Ioniq 5 AWD 73kw Premium Plus awd call Jo,https://www.donedeal.ie/cars-for-sale/hyundai-...,Premium Plus,"44,410 km",430 km (WLTP) Electric,Feb 2026,2022,Automatic,1,Hyundai,Electric,MPV,5,Luxury Trim,5,Black,120.0,Ireland,No,NaN,Franchise Dealer,"Sandyford, Co. Dublin",IONIQ 5,"€34,950",NaN,4.64 m,NaN,1.89 m,NaN,NaN,NaN,NaN,NaN,NaN,NaN,306 hp,185 km/h,5.2 sec,NaN,NaN,NaN,NaN,NaN
4,Hyundai INSTER Elegance 49kw,https://www.donedeal.ie/cars-for-sale/hyundai-...,---,0 km,Electric,---,2025,Automatic,---,NaN,Electric,Hatchback,5,---,---,---,NaN,NaN,NaN,Co. Dublin,Franchise Dealer,"Swords, Co. Dublin",INSTER,"€23,550",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:

#Here is where I combine both of the datasets
combined_df = pd.concat([df2, df3], ignore_index=True)
display(combined_df.head())




,name,url,Trim,Mileage,Engine,NCT_Expiry,Year,Transmission,Total_Owners,Make,Fuel_Type,Body_Type,Doors,Trim_Level,Seats,Colour,Road_Tax,Rural_Fuel_Consumption,Comb_Fuel_Consumption,Urban_Fuel_Consumption,Engine_Size,External_Length,External_Height,External_Width,Euro_NCAP_Overall_Safety_Rating,Child_Protection_Rating,Safety_Assistance_Rating,Adult_Protection_Rating,Pedestrian_Protection_Rating,Current_Country_Of_Reg,Imported,Independant_Dealer,Dublin2,Model,Price,Boot_Volume,Ground_Clearance,County,Fuel_Tank_Capacity,Engine_Power,Top_Speed,Battery_Capacity,Acceleration,section
0,Hyundai Tucson 2016 Executive,https://www.donedeal.ie/cars-for-sale/hyundai-...,Executive,"128,000 mi",1.7 L Diesel,Jun 2026,2016,Manual,2,Hyundai,Diesel,SUV,5,Medium Trim,5,Black,200.0,4.0 L/100 km,4.4 L/100 km,5.1 L/100 km,1.7 L,4.48 m,1.65 m,1.85 m,5/5,85%,71%,86%,71%,Ireland,No,Private Seller\nNo rating yet,"Ardara, Co. Donegal",Tucson,"€10,500",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,selection1
1,Audi Q5 40 TDI Quattro S Line 5dr S Tronic,https://www.donedeal.ie/cars-for-sale/audi-q5-...,---,"45,139 mi",2.0 L Diesel,---,2021,Automatic,---,Audi,Diesel,---,---,---,5,White,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UK,NaN,Franchise Dealer,"Dungannon, Co. Tyrone",Q5,"£27,740",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,selection1
2,Volkswagen Passat 2013 - New NCT,https://www.donedeal.ie/cars-for-sale/volkswag...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,selection1
3,2018 AUDI A6 DIESEL SALOON 2.0 TDI ULTRA S- LINE,https://www.donedeal.ie/cars-for-sale/2018-aud...,S Line,"81,300 mi",2.0 L Diesel,Jun 2026,2018,Manual,1,Audi,Diesel,Saloon,4,Sports Trim,5,Grey,200.0,3.9 L/100 km,4.4 L/100 km,5.2 L/100 km,2.0 L,4.93 m,1.46 m,1.87 m,5/5,83%,86%,91%,41%,Ireland,UK,Verified Trader\nNo rating yet,"Derry City, Co. Derry",A6,"€22,500",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,selection1
4,Mercedes-Benz E-Class E220 D Coupe AMG Sport,https://www.donedeal.ie/cars-for-sale/mercedes...,AMG SPORT,"137,000 km",2.0 L Diesel,Apr 2026,2018,Automatic,4,Mercedes-Benz,Diesel,Coupe,2,Sports Trim,4,Blue,200.0,3.7 L/100 km,4.0 L/100 km,4.5 L/100 km,2.0 L,4.83 m,1.43 m,1.86 m,5/5,90%,62%,95%,77%,Ireland,No,Independent Dealer,"Nenagh, Co. Tipperary",E-Class,"€28,950",425 L,10 cm,Co. Tipperary,NaN,NaN,NaN,NaN,NaN,selection1


In [ ]:

#Combining the make and model columns into one
combined_df['combined'] = combined_df['Make'].astype(str) + ' ' + combined_df['Model'].astype(str)
display(combined_df.head())



,name,url,Trim,Mileage,Engine,NCT_Expiry,Year,Transmission,Total_Owners,Make,Fuel_Type,Body_Type,Doors,Trim_Level,Seats,Colour,Road_Tax,Rural_Fuel_Consumption,Comb_Fuel_Consumption,Urban_Fuel_Consumption,Engine_Size,External_Length,External_Height,External_Width,Euro_NCAP_Overall_Safety_Rating,Child_Protection_Rating,Safety_Assistance_Rating,Adult_Protection_Rating,Pedestrian_Protection_Rating,Current_Country_Of_Reg,Imported,Independant_Dealer,Dublin2,Model,Price,Boot_Volume,Ground_Clearance,County,Fuel_Tank_Capacity,Engine_Power,Top_Speed,Battery_Capacity,Acceleration,section,combined
0,Hyundai Tucson 2016 Executive,https://www.donedeal.ie/cars-for-sale/hyundai-...,Executive,"128,000 mi",1.7 L Diesel,Jun 2026,2016,Manual,2,Hyundai,Diesel,SUV,5,Medium Trim,5,Black,200.0,4.0 L/100 km,4.4 L/100 km,5.1 L/100 km,1.7 L,4.48 m,1.65 m,1.85 m,5/5,85%,71%,86%,71%,Ireland,No,Private Seller\nNo rating yet,"Ardara, Co. Donegal",Tucson,"€10,500",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,selection1,Hyundai Tucson
1,Audi Q5 40 TDI Quattro S Line 5dr S Tronic,https://www.donedeal.ie/cars-for-sale/audi-q5-...,---,"45,139 mi",2.0 L Diesel,---,2021,Automatic,---,Audi,Diesel,---,---,---,5,White,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UK,NaN,Franchise Dealer,"Dungannon, Co. Tyrone",Q5,"£27,740",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,selection1,Audi Q5
2,Volkswagen Passat 2013 - New NCT,https://www.donedeal.ie/cars-for-sale/volkswag...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,selection1,nan nan
3,2018 AUDI A6 DIESEL SALOON 2.0 TDI ULTRA S- LINE,https://www.donedeal.ie/cars-for-sale/2018-aud...,S Line,"81,300 mi",2.0 L Diesel,Jun 2026,2018,Manual,1,Audi,Diesel,Saloon,4,Sports Trim,5,Grey,200.0,3.9 L/100 km,4.4 L/100 km,5.2 L/100 km,2.0 L,4.93 m,1.46 m,1.87 m,5/5,83%,86%,91%,41%,Ireland,UK,Verified Trader\nNo rating yet,"Derry City, Co. Derry",A6,"€22,500",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,selection1,Audi A6
4,Mercedes-Benz E-Class E220 D Coupe AMG Sport,https://www.donedeal.ie/cars-for-sale/mercedes...,AMG SPORT,"137,000 km",2.0 L Diesel,Apr 2026,2018,Automatic,4,Mercedes-Benz,Diesel,Coupe,2,Sports Trim,4,Blue,200.0,3.7 L/100 km,4.0 L/100 km,4.5 L/100 km,2.0 L,4.83 m,1.43 m,1.86 m,5/5,90%,62%,95%,77%,Ireland,No,Independent Dealer,"Nenagh, Co. Tipperary",E-Class,"€28,950",425 L,10 cm,Co. Tipperary,NaN,NaN,NaN,NaN,NaN,selection1,Mercedes-Benz E-Class


In [ ]:


shape = combined_df.shape

num_rows = shape[0]
num_columns = shape[1]

print(f"Number of rows: {num_rows}")
print(f"Number of columns: {num_columns}")




Number of rows: 17192
Number of columns: 44


In [ ]:
# SOME COLUMNS WERE NAMED INCORRECTLY WHEN SCRAPED FROM THE DONEDEAL WEBSITE, I RENAME THEM HERE

combined_df = combined_df.rename(columns={"Dublin2": "Counties"})

# DROPPING THE ROWS WITH NO PRICE
combined_df = combined_df.dropna(subset=['Price'])
combined_df.head()


,name,url,Trim,Mileage,Engine,NCT_Expiry,Year,Transmission,Total_Owners,Make,Fuel_Type,Body_Type,Doors,Trim_Level,Seats,Colour,Road_Tax,Rural_Fuel_Consumption,Comb_Fuel_Consumption,Urban_Fuel_Consumption,Engine_Size,External_Length,External_Height,External_Width,Euro_NCAP_Overall_Safety_Rating,Child_Protection_Rating,Safety_Assistance_Rating,Adult_Protection_Rating,Pedestrian_Protection_Rating,Current_Country_Of_Reg,Imported,Independant_Dealer,Counties,Model,Price,Boot_Volume,Ground_Clearance,County,Fuel_Tank_Capacity,Engine_Power,Top_Speed,Battery_Capacity,Acceleration,section
0,Hyundai Tucson 2016 Executive,https://www.donedeal.ie/cars-for-sale/hyundai-...,Executive,"128,000 mi",1.7 L Diesel,Jun 2026,2016,Manual,2,Hyundai,Diesel,SUV,5,Medium Trim,5,Black,200.0,4.0 L/100 km,4.4 L/100 km,5.1 L/100 km,1.7 L,4.48 m,1.65 m,1.85 m,5/5,85%,71%,86%,71%,Ireland,No,Private Seller\nNo rating yet,"Ardara, Co. Donegal",Tucson,"€10,500",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,selection1
1,Audi Q5 40 TDI Quattro S Line 5dr S Tronic,https://www.donedeal.ie/cars-for-sale/audi-q5-...,---,"45,139 mi",2.0 L Diesel,---,2021,Automatic,---,Audi,Diesel,---,---,---,5,White,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UK,NaN,Franchise Dealer,"Dungannon, Co. Tyrone",Q5,"£27,740",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,selection1
3,2018 AUDI A6 DIESEL SALOON 2.0 TDI ULTRA S- LINE,https://www.donedeal.ie/cars-for-sale/2018-aud...,S Line,"81,300 mi",2.0 L Diesel,Jun 2026,2018,Manual,1,Audi,Diesel,Saloon,4,Sports Trim,5,Grey,200.0,3.9 L/100 km,4.4 L/100 km,5.2 L/100 km,2.0 L,4.93 m,1.46 m,1.87 m,5/5,83%,86%,91%,41%,Ireland,UK,Verified Trader\nNo rating yet,"Derry City, Co. Derry",A6,"€22,500",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,selection1
4,Mercedes-Benz E-Class E220 D Coupe AMG Sport,https://www.donedeal.ie/cars-for-sale/mercedes...,AMG SPORT,"137,000 km",2.0 L Diesel,Apr 2026,2018,Automatic,4,Mercedes-Benz,Diesel,Coupe,2,Sports Trim,4,Blue,200.0,3.7 L/100 km,4.0 L/100 km,4.5 L/100 km,2.0 L,4.83 m,1.43 m,1.86 m,5/5,90%,62%,95%,77%,Ireland,No,Independent Dealer,"Nenagh, Co. Tipperary",E-Class,"€28,950",425 L,10 cm,Co. Tipperary,NaN,NaN,NaN,NaN,NaN,selection1
6,"Suzuki Swift 2017, Auto, Incredibly Low Mileage",https://www.donedeal.ie/cars-for-sale/suzuki-s...,SZ5,"27,654 km",1.2 L Petrol Hybrid,Oct 2025,2017,Automatic,3,Suzuki,Petrol Hybrid,Hatchback,5,Luxury Trim,5,Black,200.0,4.2 L/100 km,4.5 L/100 km,4.9 L/100 km,1.2 L,3.84 m,1.50 m,1.74 m,3/5,75%,25%,83%,69%,Ireland,Unknown country,Private Seller\nNo rating yet,"Greystones, Co. Wicklow",Swift,"€11,000",265 L,12 cm,NaN,NaN,NaN,NaN,NaN,NaN,selection1


In [ ]:

df3 = pd.read_csv("/content/drive/MyDrive/Reddit_Car_Posts.csv")


In [ ]:
from textblob import TextBlob

text = df3['reddit_content'].str.cat(sep=' ')
blob = TextBlob(text)
print(blob.sentiment.polarity)

0.12932882630189807


In [ ]:
import pandas as pd
from textblob import TextBlob

df3 = pd.read_csv("/content/drive/MyDrive/Reddit_Car_Posts.csv")

def get_sentiment(text):
    return TextBlob(str(text)).sentiment.polarity

df3['sentiment'] = df3['reddit_content'].apply(get_sentiment)

result = df3[['make', 'model', 'sentiment']]
print(result)


           make model  sentiment
0    Alfa Romeo   145   0.000000
1    Alfa Romeo   155   0.475000
2    Alfa Romeo   159   0.030000
3    Alfa Romeo    GT   0.264436
4    Alfa Romeo   GTV   0.400000
..          ...   ...        ...
655       Volvo   V90   0.346667
656       Volvo  XC40   0.102717
657       Volvo  XC60   0.026623
658       Volvo  XC70   0.181696
659       Volvo  XC90   0.079191

[660 rows x 3 columns]


In [ ]:
!pip install torch torchvision torchaudio
!pip install torch==1.8.1+cpu torchvision==0.9.1+cpu torchaudio==0.8.1 -f https://download.pytorch.org/whl/torch_stable.html



!pip install transformers requests beautifulsoup4 pandas numpy

from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re
!pip install transformers requests beautifulsoup4 pandas numpy
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 80.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 850.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 66.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [ ]:


tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/669M [00:00<?, ?B/s]

In [ ]:

tokens = tokenizer.encode('It was good but couldve been better. Great', return_tensors='pt')
result = model(tokens)
result.logits
int(torch.argmax(result.logits))+1

4

In [ ]:
import numpy as np
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification


df3 = df3.loc[:, ~df3.columns.str.contains('^Unnamed')]

text_column = 'reddit_content'

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt', truncation=True, max_length=512)
    result = model(tokens)
    return int(torch.argmax(result.logits))

df3['sentiment3'] = df3[text_column].astype(str).apply(lambda x: sentiment_score(x[:512]))

print(df3[[text_column, 'sentiment3']])




tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

                                        reddit_content  sentiment3
0    Dúvida sobre motor da alfa romeo 145 | Galera ...           1
1    Love that they've got the DTM bodykit for the ...           2
2    I've bought my first Alfa Romeo - The 159 SW |...           1
3    Alfa Romeo GT for first car? | I've been looki...           2
4    Alfa Romeo Gtv 1.8 spotted | If it's the 1.8, ...           1
..                                                 ...         ...
655  What are your honest thoughts on the Volvo V90...           1
656  For you Volvo XC40 owners | For you Volvo XC40...           1
657  Volvo XC60 died of natural causes after 6 year...           1
658  Is it safe to go for a 2007 Volvo XC70? | I'm ...           1
659  Talk me out of a Volvo XC90 | Keeping my eye o...           1

[660 rows x 2 columns]


In [ ]:
import numpy as np
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification


df3 = df3.loc[:, ~df3.columns.str.contains('^Unnamed')]

text_column = 'reddit_content'

tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt', truncation=True, max_length=512)
    result = model(tokens)
    return int(torch.argmax(result.logits))

df3['sentiment2'] = df3[text_column].astype(str).apply(lambda x: sentiment_score(x[:512]))

# View results
print(df3[[text_column, 'sentiment2']])


                                        reddit_content  sentiment2
0    Dúvida sobre motor da alfa romeo 145 | Galera ...           2
1    Love that they've got the DTM bodykit for the ...           5
2    I've bought my first Alfa Romeo - The 159 SW |...           4
3    Alfa Romeo GT for first car? | I've been looki...           4
4    Alfa Romeo Gtv 1.8 spotted | If it's the 1.8, ...           3
..                                                 ...         ...
655  What are your honest thoughts on the Volvo V90...           3
656  For you Volvo XC40 owners | For you Volvo XC40...           3
657  Volvo XC60 died of natural causes after 6 year...           1
658  Is it safe to go for a 2007 Volvo XC70? | I'm ...           2
659  Talk me out of a Volvo XC90 | Keeping my eye o...           4

[660 rows x 2 columns]


In [ ]:
print(df3[[text_column, 'sentiment2', 'sentiment3']])
df3.head()


                                        reddit_content  sentiment2  sentiment3
0    Dúvida sobre motor da alfa romeo 145 | Galera ...           2           1
1    Love that they've got the DTM bodykit for the ...           5           2
2    I've bought my first Alfa Romeo - The 159 SW |...           4           1
3    Alfa Romeo GT for first car? | I've been looki...           4           2
4    Alfa Romeo Gtv 1.8 spotted | If it's the 1.8, ...           3           1
..                                                 ...         ...         ...
655  What are your honest thoughts on the Volvo V90...           3           1
656  For you Volvo XC40 owners | For you Volvo XC40...           3           1
657  Volvo XC60 died of natural causes after 6 year...           1           1
658  Is it safe to go for a 2007 Volvo XC70? | I'm ...           2           1
659  Talk me out of a Volvo XC90 | Keeping my eye o...           4           1

[660 rows x 3 columns]


,make,model,reddit_content,sentiment,sentiment3,sentiment2
0,Alfa Romeo,145,Dúvida sobre motor da alfa romeo 145 | Galera ...,0.000000,1,2
1,Alfa Romeo,155,Love that they've got the DTM bodykit for the ...,0.475000,2,5
2,Alfa Romeo,159,I've bought my first Alfa Romeo - The 159 SW |...,0.030000,1,4
3,Alfa Romeo,GT,Alfa Romeo GT for first car? | I've been looki...,0.264436,2,4
4,Alfa Romeo,GTV,"Alfa Romeo Gtv 1.8 spotted | If it's the 1.8, ...",0.400000,1,3


In [ ]:
df3['combined'] = df3['make'].astype(str) + ' ' + df3['model'].astype(str)
df3.head()


,make,model,reddit_content,sentiment,sentiment3,sentiment2,combined
0,Alfa Romeo,145,Dúvida sobre motor da alfa romeo 145 | Galera ...,0.000000,1,2,Alfa Romeo 145
1,Alfa Romeo,155,Love that they've got the DTM bodykit for the ...,0.475000,2,5,Alfa Romeo 155
2,Alfa Romeo,159,I've bought my first Alfa Romeo - The 159 SW |...,0.030000,1,4,Alfa Romeo 159
3,Alfa Romeo,GT,Alfa Romeo GT for first car? | I've been looki...,0.264436,2,4,Alfa Romeo GT
4,Alfa Romeo,GTV,"Alfa Romeo Gtv 1.8 spotted | If it's the 1.8, ...",0.400000,1,3,Alfa Romeo GTV


In [ ]:
df3 = df3.drop(['make', 'model'], axis=1)
df3.head()



,reddit_content,sentiment,sentiment3,sentiment2,combined
0,Dúvida sobre motor da alfa romeo 145 | Galera ...,0.000000,1,2,Alfa Romeo 145
1,Love that they've got the DTM bodykit for the ...,0.475000,2,5,Alfa Romeo 155
2,I've bought my first Alfa Romeo - The 159 SW |...,0.030000,1,4,Alfa Romeo 159
3,Alfa Romeo GT for first car? | I've been looki...,0.264436,2,4,Alfa Romeo GT
4,"Alfa Romeo Gtv 1.8 spotted | If it's the 1.8, ...",0.400000,1,3,Alfa Romeo GTV


In [ ]:
merged_df = pd.merge(combined_df, df3, on='combined', how='inner')

merged_df.head()

merged_df = pd.merge(combined_df, df3, on='combined', how='inner')
merged_df.to_csv('merged_data.csv', index=False)
merged_df.head()

merged_df.to_csv('merged_data.csv', index=False)
files.download('merged_data.csv')




<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>